In [ ]:
#%help()

### Connecting

In [ ]:
#%connect(host=try.activeeon.com,port=8080,login=YOUR_LOGIN,password=YOUR_PASSWORD)

### Import libraries

In [ ]:
#%import()
import pandas as pd
import pickle
import bz2
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### Adding fork environment to _import_

In [ ]:
#%job_fork_env(path=./fork_env_shared.py)

### Creating task _import_

In [ ]:
#%task(name=import,export=[dataframe_json])
boston = load_boston()
dataframe_load = pd.DataFrame(boston.data)
dataframe_load.columns = boston.feature_names 
data_label = boston.target
dataframe = dataframe_load.assign(LABEL=data_label)

dataframe_json = dataframe.to_json(orient='split').encode()
compressed_data = bz2.compress(dataframe_json)
dataframe.head()

### Creating task _split_

In [ ]:
#%task(name=split,dep=[import],import=[dataframe_json],export=[X_train_json,X_test_json])
dataframe = pd.read_json(dataframe_json, orient='split')

X_train, X_test = train_test_split(dataframe, test_size=30)

X_train_json = X_train.to_json(orient='split').encode()
X_test_json = X_test.to_json(orient='split').encode()

### Creating task _train_

In [ ]:
#%task(name=train,dep=[split], import=[X_train_json], export=[filename,columns])
# Fit the model on 33%
columns="LABEL"

X_train = pd.read_json(X_train_json, orient='split')

dataframe_train = X_train.drop(columns, axis=1, inplace=False)
dataframe_label = X_train.filter(columns, axis=1)
model = LinearRegression()
model.fit(dataframe_train, dataframe_label)
# save the model to disk
filename = '/shared/finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

### Creating task _predict_

In [ ]:
#%task(name=predict,dep=[train,split], import=[columns,filename,X_test_json])
X_test = pd.read_json(X_test_json, orient='split')

dataframe_test = X_test.drop(columns, axis=1, inplace=False)
dataframe_label = X_test.filter(columns, axis=1)
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(dataframe_test, dataframe_label)

print('The prediction result is = ' + str(result))

### Plotting the job

In [ ]:
#%draw_job()

### Display ProActive portals

In [ ]:
#%show_scheduling_portal()

### Submitting the job to the scheduler

In [ ]:
#%submit_job(name=MLExample)

### Getting job results and outputs

In [ ]:
#%get_job_result()

In [ ]:
#%print_job_output()